In [1]:
#convert dataframe elements to lowercase
#removed stopwords
#lemmatization and stemming to obtain base word so as to group them together 
def preprocess(data):
    return [''.join([ss.stem(lemma.lemmatize(j))+' ' for j in str(i).split() 
                     if (j not in stpwrds) and j.isalpha()]) for i in data.str.lower()]

#calculate frequency of occurence of each word occuring in the review
def frequent(frame):
    return frame.str.split().explode().value_counts().rename_axis('words').reset_index(name='counts')

import numpy as np
import pandas as pd
import string
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.corpus import wordnet, stopwords

amazon=pd.read_csv('C:\\Users\\Pratik\\Desktop\\1611766549_6456213_train_file.dat',sep='\t',header=None,names=['Rating','Review'])
test_data=pd.DataFrame({'Review': open('C:\\Users\\Pratik\\Desktop\\1611766549_7170458_test.dat',"r")})

#drop duplicates in the training data (18506 to 18499 rows)
amazon=amazon.drop_duplicates(keep='first',inplace=False)

ss=SnowballStemmer('english')
lemma=WordNetLemmatizer()

stpwrds=stopwords.words('english')
#remove Negative words from stopwords library to retain meaning of the review
neg_words=['ain','aren',"aren't",'couldn',"couldn't",'didn',"didn't",'doesn',"doesn't",'don',"don't",'hadn',"hadn't",'hasn',
           "hasn't",'haven',"haven't",'isn',"isn't",'mightn',"mightn't",'mustn',"mustn't",'needn',"needn't",'no','nor','not',
           'shan',"shan't",'shouldn',"shouldn't",'wasn',"wasn't",'weren',"weren't",'won',"won't",'wouldn',"wouldn't"]
stpwrds=set(stpwrds)-set(neg_words)

amazon['Rev']=preprocess(amazon['Review'])
test_data['Rev']=preprocess(test_data['Review'])

#identifying frequently occurring words to be used as features for the Training and testing matrix
words=frequent(amazon['Rev'])
test_words=frequent(test_data['Rev'])

In [2]:
#term frequency–inverse document frequency vectorizer for normalizing the word count
from sklearn.feature_extraction.text import TfidfVectorizer
tvec=TfidfVectorizer(use_idf=True,vocabulary=list(set(words[(words.counts>1)]['words']) & set(test_words[(test_words.counts>1)]['words'])))
X_train=pd.DataFrame(tvec.fit_transform(amazon['Rev']).toarray(),columns=tvec.get_feature_names())
y_train=amazon['Rating']
X_test=pd.DataFrame(tvec.fit_transform(test_data['Rev']).toarray(),columns=tvec.get_feature_names())

In [ ]:
#K-fold testing
'''
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
kf=KFold(n_splits=3,random_state=None)
acc_score = []
for train_index , test_index in kf.split(X):
    X_tr , X_ts = X_train.iloc[train_index,:],X_train.iloc[test_index,:]
    y_tr , y_ts = y_train[train_index] , y_train[test_index]
    calc=X_tr.dot(X_ts.T)
    y_pred=[y_tr.iloc[calc[c].nlargest(15).index.tolist()].value_counts().idxmax() for c in calc]
    acc_score+=[accuracy_score(y_ts, y_pred)]
'''

In [ ]:
#Train Test Split validation: 80-20 split
'''
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score
X_tr,X_ts,y_tr,y_ts=train_test_split(X_train,y_train,test_size=0.2,shuffle=True)
calc=X_tr.dot(X_ts.T)
y_pred=[y_tr.iloc[calc[c].nlargest(15).index.tolist()].value_counts().idxmax() for c in calc]
acc_score=accuracy_score(y_ts, y_pred)
'''

In [ ]:
#old code for KNN without vectorization
'''
y_pred=[]
count=0
for index,i in X_test.iterrows():
    count+=1
    if count>50:
        break
    calc=pd.DataFrame((X_train*list(i)).sum(axis=1)/((((X_train**2).sum(axis=1)).multiply(sum(np.array(i)**2)))**0.5)).replace(np.nan, 0)
    calc.columns=['val']
    calc['output']=np.array(y_train)
    y_pred+=[(calc.nlargest(15,['val']))['output'].value_counts().idxmax()]
'''

In [3]:
#Cosine Similarity calculate (ignoring common denominator)
#(X_train*list(i)).sum(axis=1)/((((X_train**2).sum(axis=1)).multiply(sum(np.array(i)**2)))**0.5)
calc=X_train.dot(X_test.T)

In [ ]:
#KNN: choosing 15 nearest neighbors having max values of cosine similarity
#obtain +1/-1 from corresponding y_train
#majority vote using idxmax
y_pred=[y_train.iloc[calc[c].nlargest(15).index.tolist()].value_counts().idxmax() for c in calc]

In [7]:
#Output file
pd.DataFrame(y_pred).to_csv(path_or_buf='C:\\Users\\Pratik\\Desktop\\submission.txt',header=False,index=False,index_label=None,mode='w') 